In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [26]:
df = pd.read_csv('lemmatized_df.csv', index_col=0).dropna()
df_test = pd.read_csv('lemmatized_tesla_tweets_df.csv', index_col=0, parse_dates=True).dropna()
df

,text,sentiment
0,-PRON- would have respond if -PRON- be go,neutral
1,Sooo SAD -PRON- will miss -PRON- here in San D...,negative
2,-PRON- boss be bully -PRON-,negative
3,what interview leave -PRON- alone,negative
4,son of INSULT why could not -PRON- put -PRON- ...,negative
...,...,...
27476,wish -PRON- could come see u on Denver husba...,negative
27477,-PRON- have wonder about rake to the client ...,negative
27478,Yay good for both of -PRON- enjoy the break ...,positive
27479,but -PRON- be worth -PRON- INSULT,positive


In [27]:
df_test

,created_at,timezone,language,text
2,2021-02-20 23:48:03 IST,200,en,to think of Bitcoin as just a currency be to t...
3,2021-02-20 23:47:55 IST,200,en,look at the environmental cost .
4,2021-02-20 23:47:38 IST,200,en,supercruise be not well than original autopi...
5,2021-02-20 23:47:37 IST,200,en,do -PRON- think the high energy use of the dep...
7,2021-02-20 23:47:15 IST,200,en,"-PRON- be so good to have -PRON- with -PRON- ,..."
...,...,...,...,...
126819,2021-02-16 18:21:25 IST,200,en,"what about Dogecoin , chief"
126821,2021-02-16 18:21:24 IST,200,en,just ask -PRON- the question through here . -P...
126822,2021-02-16 18:21:24 IST,200,en,Can -PRON- speak to the blackout in Texas ? Pe...
126823,2021-02-16 18:21:24 IST,200,en,careful of the black ice . not worth drive on ...


In [5]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
from sklearn.model_selection import train_test_split

tweet_df_train, tweet_df_test = train_test_split(df, test_size=0.1)

In [10]:
import tensorflow as tf



def tweet_gen(df):
    def get_sentiment_label(row):
        if row.sentiment == 'positive':
            return 2
        if row.sentiment == 'neutral':
            return 1
        if row.sentiment == 'negative':
            return 0
    
    def g():
        for row in df.itertuples():
            text = row.text
            label = get_sentiment_label(row)
            tokenized = tokenizer( row.text, max_length=280, padding='max_length', truncation=True )
            yield {k: np.array(tokenized[k]) for k in tokenized}, label
    return g

input_names = ['input_ids', 'token_type_ids', 'attention_mask']
data_types = ({k: tf.int32 for k in input_names}, tf.int64)
data_shapes = ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([]))

tweet_data_train = tf.data.Dataset.from_generator(
    tweet_gen(tweet_df_train),
    data_types, data_shapes
).shuffle(100).batch(32).repeat(4)

tweet_data_test = tf.data.Dataset.from_generator(
    tweet_gen(tweet_df_test),
    data_types, data_shapes
).shuffle(100).batch(32).repeat(4)

In [11]:
list(tweet_data_train.take(1))

[({'input_ids': <tf.Tensor: shape=(32, 280), dtype=int32, numpy=
   array([[  101, 10680,  2022, ...,     0,     0,     0],
          [  101,  2748,  1011, ...,     0,     0,     0],
          [  101,  2022, 13299, ...,     0,     0,     0],
          ...,
          [  101,  5292,  9397, ...,     0,     0,     0],
          [  101,  1045,  2031, ...,     0,     0,     0],
          [  101,  1011,  4013, ...,     0,     0,     0]], dtype=int32)>,
   'token_type_ids': <tf.Tensor: shape=(32, 280), dtype=int32, numpy=
   array([[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
   'attention_mask': <tf.Tensor: shape=(32, 280), dtype=int32, numpy=
   array([[1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          ...,
          [1, 1, 1, ..., 0, 0, 0],
          [1, 

In [12]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from transformers import TFBertForSequenceClassification

tweet_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

tweet_model.compile(optimizer=Adam(lr=3e-5),
                   loss=SparseCategoricalCrossentropy(from_logits=True),
                   metrics='accuracy')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
tweet_model.fit(tweet_data_train,
                validation_data=tweet_data_test,
                epochs=10,
                steps_per_epoch=64,
                validation_steps=16)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7ffced6a94f8> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7ffced6a94f8> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


KeyboardInterrupt: 

In [ ]:
tweet_model.predict()

In [18]:
df_live = pd.read_csv("../live_tweets_apple.csv", delimiter="	", parse_dates=True)

In [21]:
df_live.created_at.min()

'2021-02-15 08:57:13 IST'

In [22]:
df_live.created_at.max()

'2021-02-20 16:10:22 IST'

In [24]:
pd.set_option('display.max_columns', 500)
df_live

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1363128866483695616,1363113169426743297,2021-02-20 16:10:22 IST,2021-02-20,16:10:22,200,2778718461,iamquent2,Quent2🫀❄️,NaN,@BooskaBlc @Lee_QLF @Apple Haahahaha,tl,[],[],[],0,0,0,[],[],https://twitter.com/iamquent2/status/136312886...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'BooskaBlc', 'name': 'dD \U00...",NaN,NaN,NaN,NaN
1,1363128788746342404,1363128788746342404,2021-02-20 16:10:03 IST,2021-02-20,16:10:03,200,1261489741029138433,ltafoyaofficial,serpent,NaN,@Apple what’s the point of Live Photos if all ...,en,[],[],[],0,0,0,[],[],https://twitter.com/ltafoyaofficial/status/136...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1363128775605710853,1363113169426743297,2021-02-20 16:10:00 IST,2021-02-20,16:10:00,200,1347327105911189504,booskablc,dD 🫀,NaN,@Lee_QLF @iamquent2 @Apple j’crois il t’as oub...,fr,[],['https://www.paypal.me/jldetraque'],[],1,0,0,[],[],https://twitter.com/BooskaBlc/status/136312877...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Lee_QLF', 'name': '🪔mod', 'i...",NaN,NaN,NaN,NaN
3,1363128695851016195,1363128693770645509,2021-02-20 16:09:41 IST,2021-02-20,16:09:41,200,1035958701599989761,mlmario03,Mario,NaN,"Que ""prendiera"" mi tarjeta para que pudiera re...",es,"[{'screen_name': 'invexbanco', 'name': 'invex ...",[],[],1,0,0,[],[],https://twitter.com/mlmario03/status/136312869...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1363128670781665284,1362832156897198080,2021-02-20 16:09:35 IST,2021-02-20,16:09:35,200,161158217,snoozealarm13,George K.,NaN,@tristannorman @9to5mac @ChanceHMiller @Apple ...,en,[],[],[],0,0,0,[],[],https://twitter.com/snoozealarm13/status/13631...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'tristannorman', 'name': 'Tri...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19187,1361208393050955777,1361208393050955777,2021-02-15 08:59:05 IST,2021-02-15,08:59:05,200,337415993,pcflwtrs,Tasha s,NaN,@Apple @elonmusk @BillGates @melindagates #...,und,"[{'screen_name': 'elonmusk', 'name': 'elon mus...",['https://vm.tiktok.com/ZMeLdC4ug/'],[],0,0,0,['tiktok'],[],https://twitter.com/PcflWtrs/status/1361208393...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
19188,1361208386897866752,1361088990120583168,2021-02-15 08:59:04 IST,2021-02-15,08:59:04,200,860084122349207552,sky_galaxy_700,🐇たわむれチノ🐇,NaN,@Bomb_Hanan @waxaxa132 @Mof_htj @hikakin @Seik...,und,[],[],[],1,0,2,[],[],https://twitter.com/sky_galaxy_700/status/1361...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Bomb_Hanan', 'name': '💣はなん🧨'...",NaN,NaN,NaN,NaN
19189,1361208264646479875,1361204877565648898,2021-02-15 08:58:34 IST,2021-02-15,08:58:34,200,1301245036961570816,mikeisback10,Mike Is Back,NaN,@timesofindia @TOIBusiness @TwitterIndia @Goog...,und,"[{'screen_name': 'google', 'name': 'google', '...",[],[],0,0,1,['breakingnews'],[],https://twitter.com/MikeIsBack10/status/136120...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'timesofindia', 'name': 'The ...",NaN,NaN,NaN,NaN
19190,1361208124179247105,1361208124179247105,2021-02-15 08:58:01 IST,2021-02-15,08:58:01,200,1161579619511754752,stunninbrina,millie 🧸,NaN,@apple we need a beige &amp; heart emoji,en,[],[],[],0,0,1,[],[],https://twitter.com/stunninbrina/status/136120...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
